In [165]:
import pandas as pd
import numpy as np
from datetime import timedelta

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import psar,zigzag




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [166]:
klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-06-20.DataFrame'
klines_file  = './backtest/klines/1h01/Full_BTCUSDT_1h01_2019-01-01_2024-06-20.DataFrame'

klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/0m15/Completo_BTCUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [ ]:
def pred_price(window,fwd):
    
    x_pred = len(window) + fwd - 1

    x = np.arange(len(window))  # Índice temporal de la ventana
    y = window.values  # Valores de 'price' en la ventana
    
    # Ajuste de grado 1 para la pendiente
    a, b = np.polyfit(x, y, 1)

    pred = x_pred * a + b

    return pred    

# Aplicamos rolling().apply() sobre `price` y unimos al DataFrame

window = 50
fwd = 3
profit = 8
stop = 8
gap = 0.5/100
df['pred'] = df['close'].rolling(window).apply(lambda x: pred_price(x,fwd))
df['change']  = ((df['pred']/df['pred'].shift(1))-1)

df['price'] = df['close']
#df['acum'] = np.where((df['high_pred']>df['close']) & (df['close']>df['low_pred']),df['close'],None)
df['trend+'] = np.where(df['change']>gap,df['pred'],None)
df['trend-'] = np.where(df['change']<-gap,df['pred'],None)

df['buy']  = np.where(df['trend+']<df['close'],df['close'],None)
df['sell'] = None#np.where(df['trend+']>df['close'],df['close'],None)

In [170]:
to_show = df#[100:550]
indicators = [
     #{'col': 'high_pred','name': 'high_pred','color': 'green','row': 1, 'mode':'lines',},
     #{'col': 'low_pred','name': 'low_pred','color': 'red','row': 1, 'mode':'lines',},
     #{'col': 'pred','name': 'pred','color': 'white','row': 1, 'mode':'lines',},
     #  {'col': 'trend+','name': 'trend+','color': 'green','row': 1, 'mode':'markers',},
       {'col': 'trend-','name': 'trend-','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_trigger','name': 'ST Trigger','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     {'col': 'change','name': 'change','color': 'white','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        {'df':to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
        {'df':to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
     ]
#if 'volume' in df.columns:
#        df.drop(columns=['volume'],inplace=True)

fig = fn.ohlc_chart(to_show,indicators=indicators,indicators_out = indicators_out, events=events) #
fig.show()